In [2]:
import pandas as pd


In [3]:
protein_type = "globular_proteins"
genes_df = pd.read_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\Data\\{protein_type}_data.csv")
genes_list = genes_df["gene"].unique().tolist()
genes_list

['SLC44A4',
 'KCNE1',
 'CISD2',
 'TMEM126A',
 'MET',
 'KCNJ10',
 'LHFPL5',
 'KCNQ1',
 'FGFR1',
 'ILDR1',
 'CHSY1',
 'BCS1L',
 'ANKH',
 'OTOF',
 'SLC33A1',
 'P2RX2',
 'TMC1',
 'GJB2',
 'KCNQ4',
 'TMEM132E',
 'KITLG',
 'SLC26A4',
 'SLC52A3',
 'GJB3',
 'FGFR3',
 'WFS1',
 'CACNA1D',
 'FGFR2',
 'SLC22A4',
 'IFNLR1',
 'SLC26A5',
 'SLC17A8',
 'OPA1',
 'SLC19A2',
 'ROR1',
 'PCDH15',
 'TMPRSS3',
 'PEX26',
 'CLDN14',
 'GREB1L',
 'ATP2B2',
 'BSND',
 'SLC52A2',
 'CLRN1',
 'LRTOMT',
 'MARVELD2',
 'S1PR2',
 'ATP6V0A4',
 'TMIE',
 'PTPRQ',
 'ABHD12',
 'CLIC5',
 'CD164',
 'CLDN9',
 'FITM2',
 'DCAF17',
 'ADCY1',
 'SLITRK6',
 'MPZL2',
 'SYNE4',
 'SPNS2',
 'SLC4A11']

In [9]:
# Get AlphaMissense predictions for all genes in the list
import csv
from data_retrievel_and_feature_extraction import uniprot_info as uni

# Create a dictionary of gene names and uniprot ids
gene_uniprot_dict = {}
for gene in genes_list:
    uniprot_id = uni.get_uniprot_id(gene)
    gene_uniprot_dict[gene] = uniprot_id
# Create a csv file with the gene names and uniprot ids
with open(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\Data\\gene_uniprot_list_{protein_type}.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["gene", "uniprot_id"])
    for key, value in gene_uniprot_dict.items():
        writer.writerow([key, value])

# # get uniprot_ids from the file in the data folder
# uniprot_ids_df = pd.read_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\Data\\gene_uniprot_list_{protein_type}.csv")
# uniprot_ids = uniprot_ids_df["uniprot_id"].tolist()

In [10]:
# Read the file and remove genes that are not in the list
tsv_file_path = 'C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\AlphaMissense\\AlphaMissense_aa_substitutions.tsv'
csv_file_path = f'C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\protein_type_predictors\\am_genes_in_{protein_type}.csv'

uniprot_ids = gene_uniprot_dict.values()

# convert the tsv file to dataframe
am_df = pd.read_csv(tsv_file_path, sep='\t')
# write a new dataframe with only the genes in the uniprot_ids list
am_df = am_df[am_df['uniprot_id'].isin(uniprot_ids)]
am_df
    

,uniprot_id,protein_variant,am_pathogenicity,am_class
10801387,O00400,M1A,0.5207,ambiguous
10801388,O00400,M1C,0.3580,ambiguous
10801389,O00400,M1D,0.9273,pathogenic
10801390,O00400,M1E,0.8093,pathogenic
10801391,O00400,M1F,0.2116,benign
...,...,...,...,...
208740456,Q9Y276,R419S,0.1633,benign
208740457,Q9Y276,R419T,0.1120,benign
208740458,Q9Y276,R419V,0.1726,benign
208740459,Q9Y276,R419W,0.1391,benign


In [4]:
# Combine the prediction files into one file
my_prediction = pd.read_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\predictions_vs_real\\{protein_type}\\predictions_vs_real_{protein_type}.csv")
my_prediction

,gene,variant,prediction,reality
0,POLR1D,L111P,1,1
1,POLR1D,T50I,1,1
2,POLR1D,L55V,1,1
3,POLR1D,G99S,1,1
4,POLR1D,C68W,1,1
...,...,...,...,...
2306,SERPINB6,V346I,0,0
2307,SERPINB6,G153S,1,0
2308,SERPINB6,M90V,1,0
2309,BDP1,V275I,0,0


In [ ]:
# Build a dataframe with all the predictions from AlphaMissense, for all genes in list genes.
am_predictions = pd.DataFrame()
for gene in genes_list:
    gene_predictions = pd.read_csv(f"C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\AlphaMissense\\am_predictions_per_gene\\am_predictions_{gene}.csv")
    am_predictions = pd.concat([am_predictions, gene_predictions])
am_predictions

In [ ]:
# Change column names to match the other file
am_predictions = am_predictions.rename(columns={"protein_variant": "variant"})

In [ ]:
# print all rows with Nan in AM score
am_vus = am_predictions[am_predictions['am_class'] == "ambiguous"]

In [ ]:
# Delete all rows with ambiguous in AF score
am_predictions = am_predictions[~(am_predictions['am_class'] == "ambiguous")]
am_predictions

In [ ]:
# Make "benign" and "pathogenic" to 0 and 1
am_predictions.loc[am_predictions['am_class'] == "benign", 'am_class'] = 0
am_predictions.loc[am_predictions['am_class'] == "pathogenic", 'am_class'] = 1

In [ ]:
# print how many pathogenic and benign variants are in the EVE file
print(f"Number of pathogenic variants in am: {len(am_predictions[am_predictions['am_class'] == 1])}")
print(f"Number of benign variants in am: {len(am_predictions[am_predictions['am_class'] == 0])}")

In [ ]:
# Merge the two files
merged = pd.merge(my_prediction, am_predictions, on=["gene", "variant"])
merged


In [ ]:
# To make sure that all tools will be tested on the same variants, save the merged with REVEL file and do so for all other tools. Then merge all the files together.
merged_only_am_columns = merged[['gene', 'variant', 'am_pathogenicity', 'am_class']]
merged.to_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\benchmarking\\alphamissense_on_dvd_data_predictions_LOPO.csv")

In [ ]:
print(f"length of merged {len(merged)}")
print(f"length of my_prediction {len(my_prediction)}")
print(f"length of alpha_missense {len(am_predictions)}")

In [ ]:
#############################################################################################################
# All next rows were used to calculate the MCC, before realised I must apply first the threshold and remove the VUS variants. Now all these rows aren't in use, since I'm doing the calculations after removing the VUS variants and with all tools together.
#############################################################################################################

In [ ]:
# ## Calculate the MCC for the mutpred predictions, according to the real pathogenicity (0 or 1)
# from sklearn.metrics import matthews_corrcoef

In [ ]:
# # separate the merged dataframe according to gene
# genes = merged["gene"].unique()
# print(f"Number of genes: {len(genes)}")

In [ ]:
# # Calculate MCC for each gene specific predictor for EVE
# genes = merged_without_nan["gene"].unique()
# print(genes)
# 
# ### EVE ####
# # Build dictionary with gene names as keys.
# mccs = {gene: 0 for gene in genes}
# 
# for gene in genes:
#     gene_df = merged_without_nan[merged_without_nan["gene"] == gene]
#     # Assuming you have a DataFrame called 'data' with 'prediction' and 'MutPred_score' columns
#     gene_df['EVE_scores_ASM'] = gene_df['EVE_scores_ASM'].astype(float)
#     # Create binary predictions based on the 0.5 threshold
#     gene_df.loc[:, 'binary_prediction_EVE'] = (gene_df['EVE_scores_ASM'] > 0.5).astype(int)
# 
#     gene_df['predictions'] = gene_df['predictions'].astype(int)
# 
#     # Calculate MCC
#     mcc = matthews_corrcoef(gene_df['predictions'], gene_df['binary_prediction_EVE'])
# 
#     # Get gene name for the use for the dictionary
#     gene = gene_df['gene'].unique()[0]
# 
#     # Append mcc to dictionary
#     mccs[gene] = mcc
# 
# print("MCCs of EVE predictions for each gene:")
# for gene in mccs:
#     print(f"{gene}: {mccs[gene]}")

In [ ]:
# from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, average_precision_score, f1_score, log_loss, confusion_matrix
# # store the results in a dictionary
# results = {gene: 0 for gene in genes}
# 
# for gene in genes:
#     gene_df = merged_without_nan[merged_without_nan["gene"] == gene]
#     gene_df['EVE_scores_ASM'] = gene_df['EVE_scores_ASM'].astype(float)
#     y_true = gene_df["pathogenicity"].astype(int)
#     predicted_probabilities = gene_df["EVE_scores_ASM"]
#     gene_df.loc[:, 'binary_prediction_EVE'] = (gene_df['EVE_scores_ASM'] > 0.5).astype(int)
#     predicted_labels = gene_df["binary_prediction_EVE"]
#     
#     results_gene = {}
#     # Calculate AUC-ROC
#     roc_auc = roc_auc_score(y_true, predicted_probabilities)
#     # append to dictionary
#     results_gene["roc_auc"] = roc_auc
#     
#     # Calculate ROC curve
#     fpr, tpr, thresholds = roc_curve(y_true, predicted_probabilities)
#     roc_auc = auc(fpr, tpr)
#     # append to dictionary
#     results_gene["auc"] = roc_auc
#     
#     # Calculate AUC-PR
#     precision, recall, _ = precision_recall_curve(y_true, predicted_probabilities)
#     pr_auc = auc(recall, precision)
#     # append to dictionary
#     results_gene["pr_auc"] = pr_auc
#     
#     # Calculate F1 Score
#     f1 = f1_score(y_true, predicted_labels)
#     # append to dictionary
#     results_gene["f1"] = f1
#     
#     # Calculate Log Loss
#     logloss = log_loss(y_true, predicted_probabilities)
#     # append to dictionary
#     results_gene["logloss"] = logloss
#     
#     # Calculate confusion matrix
#     conf_matrix = confusion_matrix(y_true, predicted_labels)
#     # append to dictionary
#     results_gene["confusion_matrix"] = conf_matrix
#     
#     # Get gene name for the use for the dictionary
#     gene = gene_df['gene'].unique()[0]
#     # Append results to dictionary
#     results[gene] = results_gene
#     
# ## Print results
# print("Results of EVE predictions for each gene:")
# for gene in results:
#     print(f"{gene}: {results[gene]}")

In [ ]:
# # present the confusion matrix for each gene
# import matplotlib.pyplot as plt
# 
# for gene in results:
#     # Plot confusion matrix
#     plt.imshow(results[gene]["confusion_matrix"], cmap=plt.cm.Blues)
#     plt.xlabel("Predicted labels")
#     plt.ylabel("True labels")
#     plt.xticks([0, 1], ["Benign", "Pathogenic"])
#     plt.yticks([0, 1], ["Benign", "Pathogenic"])
#     plt.title(f"Confusion matrix for {gene}, EVE predictions")
#     plt.style.use("seaborn-white")
#     # Add text annotations
#     for i in range(2):
#         for j in range(2):
#             plt.text(j, i, results[gene]["confusion_matrix"][i, j], ha="center", va="center", color="black", backgroundcolor="white")
#     plt.colorbar()
#     plt.show()

In [ ]:
# present the ROC curve for each gene
# for gene in results:
#     # Plot ROC curve
#     plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {results[gene]['roc_auc']:.2f})")
#     plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel("False Positive Rate")
#     plt.ylabel("True Positive Rate")
#     plt.title(f"ROC curve for {gene}, MutPred2 predictions")
#     plt.legend(loc="lower right")
#     plt.show()